Lược đồ mã hóa & giải mã ECIES

Nguồn: https://github.com/sara62/cryptosage

In [ ]:
# Cần load các file
#load('cryptosage/prime192v1.sage')
#load('cryptosage/digest.sage')
#load('cryptosage/mathhelper.sage')

In [7]:
# elliptic curve domain parameters, prime192v1
#
# p = 2^192 - 2^64 - 1
# a = -3
# b = 0x64210519E59C80E70FA7E9AB72243049FEB8DEECC146B9B1
# x = 0x188DA80EB03090F67CBF20EB43A18800F4FF0AFD82FF1012
# y = 0x07192B95FFC8DA78631011ED6B24CDD573F977A11E794811
# n = 0xFFFFFFFFFFFFFFFFFFFFFFFF99DEF836146BC9B1B4D22831
# h = 1
#

F = FiniteField(2^192 - 2^64 - 1)
a  = -3
b  = 0x64210519E59C80E70FA7E9AB72243049FEB8DEECC146B9B1
E  = EllipticCurve(F, [a, b])
P  = E((0x188DA80EB03090F67CBF20EB43A18800F4FF0AFD82FF1012,
	0x07192B95FFC8DA78631011ED6B24CDD573F977A11E794811))
n  = 0xFFFFFFFFFFFFFFFFFFFFFFFF99DEF836146BC9B1B4D22831
h  = 1
Fn = FiniteField(n)

In [8]:
Fn

Finite Field of size 6277101735386680763835789423176059013767194773182842284081

In [11]:
# Algorithm 4.24 Elliptic curve key pair generation
# Require:
#	generator point P of elliptic curve E
#	order n of P and the field Zn defined by n
# Input:
#	N/A
# Output:
#	keypair (Q, d)
#		public key point Q on curve E
#		private key d in [1, n-1]
#
def ec_keygen():
    print("keygen")
    d = randint(1, n - 1)
    Q = d * P
    return (Q, d)

In [12]:
ec_keygen()

keygen


((3219111610613358183542838271671416630398847458863477932913 : 871633723302901091515929119260834933527254961529684842778 : 1),
 871519068760295588806893242525372809390482873792227098369)

In [4]:
# Algorithm 4.42 ECIES encryption
# Require:
# generator point P of elliptic curve E
# order n of P and the field Zn defined by n
# Input:
#  message m
#  public key Q
# Output:
# Ciphertext (R,C,t).
#

def ecies_encrypt(Q, m):
    k = randint(1, n - 1)
    R = k * P
    Z = h * k * Q
    l = ceil(math.log(n, 2)/8)
    (zx, zy) = Z.xy()
    zxstr = I2OSP(zx, l)
    rstr = point2str(R, l)
    str = KDF(zxstr, l, rstr)
    k1 = str[0:l]
    k2 = str[l:2*l]
    print(k1)
    aesobj = AES.new(k1)
    C = aesobj.encrypt(pad(m))
    hmacobj = hmac.new(k2)
    hmacobj.update(C)
    t = hmacobj.digest()
    print(t)
    return [R, C, t]

In [13]:
(Q, d) = ec_keygen()

keygen


In [14]:
Q

(6228556327555745807561503616435220683988145186967202486841 : 3267570400187931883573360604617139155976253398630511539563 : 1)

In [15]:
d

2915942612306015850552618892716344743363107847870191726739

In [16]:
#pk: Q, sk:d
m = 'hello'

In [17]:
# ECIES
[R, C, t] = ecies_encrypt(Q, m)

NameError: name 'I2OSP' is not defined

In [5]:
# Algorithm 4.43 ECIES decryption
# Require:
#   generator point P of elliptic curve E
#   order n of P and the field Zn defined by n
# Input:
#   private key d
#   ciphertext (R,C,t)
# Output:
#   Plaintext m or rejection of the ciphertext.
#

def ecies_decrypt(R, C, t, d):
    Z = h * d * R
    l = ceil(math.log(n, 2)/8)
    (zx, zy) = Z.xy()
    zxstr = I2OSP(zx, l)
    rstr = point2str(R, l)
    str = KDF(zxstr, l, rstr)
    k1 = str[0:l]
    k2 = str[l:2*l]
    hmacobj = hmac.new(k2)
    hmacobj.update(C)
    t1 = hmacobj.digest()
    print(t1)
    if t1 != t:
        return False
    aesobj = AES.new(k1)
    m = aesobj.decrypt(C)
    return m